http://wonder.cdc.gov/wonder/help/mcd.html#2014-Revision.

In [78]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import numpy as np
from opendata_key import g_key
from opendata_key import open_key 
import requests
import gmaps
import time


In [ ]:
open_key

In [79]:
base_url = "https://public.opendatasoft.com/api/records/1.0/search"
params = {'api_key': open_key,
'dataset': 'us-mass-shootings-1982-2017',
'facet': 'location',
'facet': 'city',
'facet': 'state',
'facet': 'weapons_obtained_legally',
'facet': 'date',
'facet': 'type_of_weapons',
'facet': 'preior_signs_of_mental_health_issues'
}


resp = requests.get(base_url, params = params).json()
resp

{'nhits': 100,
 'parameters': {'dataset': 'us-mass-shootings-1982-2017',
  'timezone': 'UTC',
  'rows': 10,
  'format': 'json',
  'facet': ['preior_signs_of_mental_health_issues']},
 'records': [{'datasetid': 'us-mass-shootings-1982-2017',
   'recordid': '22ece442447f032a89ad562fa848b4e59cf4d098',
   'fields': {'injured': 3,
    'case': 'R.E. Phelon Company shooting',
    'mental_health_details': 'An ex-con, he had been freed from prison, although he displayed violent tendencies.',
    'race': 'Black',
    'total_victims': 7,
    'weapons_obtained_legally': 'No',
    'venue': 'Workplace',
    'type_of_weapons': 'One semiautomatic handgun',
    'coordinates': [33.5598586, -81.721952],
    'summary': 'Ex-con Hastings Arthur Wise, 43, opened fire at the R.E. Phelon Company in retaliation for being fired after an argument with a supervisor. He attempted suicide by ingesting insecticide, failed, and was executed by the state of South Carolina eight years later.',
    'sources': 'http://www.

In [102]:
xx=resp['records'][0]['fields']['state'][:]
xx

'SouthCarolina'

In [80]:
filter_col = [col for col in pd.json_normalize(resp['records']) if col.startswith('fields')]
filter_col

['fields.injured',
 'fields.case',
 'fields.mental_health_details',
 'fields.race',
 'fields.total_victims',
 'fields.weapons_obtained_legally',
 'fields.venue',
 'fields.type_of_weapons',
 'fields.coordinates',
 'fields.summary',
 'fields.sources',
 'fields.state',
 'fields.mental_health_sources',
 'fields.prior_signs_of_mental_health_issues',
 'fields.location',
 'fields.gender',
 'fields.date',
 'fields.fatalities',
 'fields.where_obtained',
 'fields.type',
 'fields.weapon_details']

In [81]:
data_set = pd.json_normalize(resp['records'])[filter_col]
data_set = data_set.dropna(how='any', axis=0).reset_index(drop=True)
data_set.to_csv("data_in/mass_shooting.csv") 
data_set

,fields.injured,fields.case,fields.mental_health_details,fields.race,fields.total_victims,fields.weapons_obtained_legally,fields.venue,fields.type_of_weapons,fields.coordinates,fields.summary,...,fields.state,fields.mental_health_sources,fields.prior_signs_of_mental_health_issues,fields.location,fields.gender,fields.date,fields.fatalities,fields.where_obtained,fields.type,fields.weapon_details
0,3.0,R.E. Phelon Company shooting,"An ex-con, he had been freed from prison, alth...",Black,7.0,No,Workplace,One semiautomatic handgun,"[33.5598586, -81.721952]","Ex-con Hastings Arthur Wise, 43, opened fire a...",...,SouthCarolina,http://chronicle.augusta.com/stories/1997/09/1...,No,Aiken,Male,1997-09-15,4.0,Unknown,Mass,9mm semiautomatic handgun
1,1.0,Fort Lauderdale revenge shooting,Co-workers complained about his threats and ve...,Black,7.0,Yes,Workplace,"One semiautomatic handgun,one revolver","[26.119269, -80.104119]","Fired city park employee Clifton McCree, 41, o...",...,Florida,http://articles.sun-sentinel.com/1996-02-11/ne...,Yes,Fort Lauderdale,Male,1996-02-09,6.0,Unknown,Spree,9mm Glock semiautomatic handgun; .32-caliber r...
2,2.0,San Francisco UPS shooting,"Lam had a history of domestic, work conflict",Asian,5.0,No,Workplace,two handguns,"[37.765947, -122.406087]","Jimmy Lam, 38, fatally shot three coworkers an...",...,California,http://www.ktvu.com/news/ktvu-local-news/26177...,Yes,San Francisco,Male,2017-06-14,3.0,Unclear; the firearm was stolen in Utah. A sec...,Mass,MAC-10-style “assault pistol”; 30-round magazi...
3,8.0,Washington Navy Yard shooting,Had told Rhode Island police the prior month t...,Black,20.0,Yes,Military,"Sawed-off shotgun,2 boxes of shells; also a .4...","[38.874981, -76.99453]","Aaron Alexis, 34, a military veteran and contr...",...,D.C.,http://bigstory.ap.org/article/13-killed-washi...,Yes,Washington,Male,2013-09-16,12.0,Sharpshooters Small Arms Range,Mass,Remington 870 Express 12-gauge shotgun; Berett...
4,1.0,Coffee shop police killings,"He had a history of erratic, bizarre behavior....",Black,5.0,No,Other,One semiautomatic handgun,"[47.15277, -122.467308]","Maurice Clemmons, 37, a felon who was out on b...",...,Washington,http://seattletimes.com/html/localnews/2010385...,Yes,Parkland,Male,2009-11-29,4.0,Stolen from an individual in Seattle.,Mass,9mm Glock 17 semiautomatic handgun; .38-calibe...
5,14.0,Stoneman Douglas High School shooting,Cruz had a long history of behavioral problems...,White,31.0,Yes,School,semiautomatic rifle,"[26.30483, -80.269511]","Nikolas J. Cruz, 19, heavily armed with an AR-...",...,Florida,https://www.nytimes.com/2018/02/15/us/nikolas-...,Yes,Parkland,Male,2018-02-14,17.0,A Florida pawn shop,Mass,AR-15
6,9.0,Planned Parenthood clinic,The judge in the case has not publicly release...,White,12.0,Unknown,Workplace,Long gun,"[38.881031, -104.849057]","Robert Lewis Dear, 57, shot and killed a polic...",...,Colorado,-,Unclear,Colorado Springs,Male,2015-11-27,3.0,Unclear,Mass,Reportedly an AK-47 style semiautomatic rifle ...
7,4.0,Binghamton shootings,He apparently harbored growing paranoia toward...,Asian,18.0,Yes,Other,Two semiautomatic handguns,"[42.099802, -75.917723]","Jiverly Wong, 41, opened fire at an American C...",...,NewYork,http://www.nytimes.com/2009/04/12/nyregion/12b...,Yes,Binghamton,Male,2009-04-03,14.0,"Gander Mountain in Johnson City, N.Y.",Mass,"9mm Beretta, .45-caliber Springfield semiautom..."
8,20.0,Texas First Baptist Church massacre,"Kelley had a history of domestic violence, inc...",White,46.0,Kelley passed federal criminal background chec...,Religious,semiautomatic rifle,"[29.273282, -98.056488]","Devin Patrick Kelley, a 26-year-old ex-US Air ...",...,Texas,http://www.expressnews.com/news/local/article/...,Yes,Sutherland Springs,Male,2017-11-05,26.0,Purchased in April 2016 from an Academy Sports...,Mass,Ruger AR-556; Kelley also possessed semiautoma...


In [87]:
clean_data = data_set.loc[:, data_set['field.date', 'field.location', 'field.state', 'field.coordinates', 'field.fatalities', 'field.injured', 'fields.type_of_weapons']]
clean_data

KeyError: ('field.date', 'field.location', 'field.state', 'field.coordinates', 'field.fatalities', 'field.injured', 'fields.type_of_weapons')

In [111]:
states = ['Texas', 'California']
city_name = []
lat = []
lng = []
state_name = []
number_killed = []
number_wounded = []
type_of_gun = []
date = []
for state in states:
# for item in range(0, len(resp['records'])):
    resp = requests.get(base_url, params = params).json()
    try:
        # city_name.append(resp[records][:]['fields'][:]['city'][:])
        lat.append(resp['records'][0:]['fields']['coordinates'][0])
        # lng.append(resp['records']['fields']['longitude'])
        # state_name.append(resp['records']['fields'][0:]['state'][:])
        # number_killed.append(resp['records']['fields']['total_number_of_fatalities'])
        # number_wounded.append(resp['records']['fields']['number_of_victims_injured'])
        # type_of_gun.append(resp['records']['fields']['type_of_gun_detailed'][:])
        # date.append(resp['records']['fields']['date'])
        # country_id.append(resp['records']['fields']['city'])
        # print(f"Processing Record {counter_record} of set {counter_set} | {city}")
    except (KeyError, IndexError):
        print("City not found. Skipping...")
print(lat)
# print(state_name)

TypeError: list indices must be integers or slices, not str

In [ ]:
    
    city_list = []
for rec in response:
    resp = requests.get(base_url, params = params)
    response = resp.json()
    city_list.append(response['records']['fields']['city'])


In [ ]:
shooting_dic = {
    'date': '',
    'city': '',
    # 'State': '',
    # 'Latitude': '',
    # 'Longitude': '',
    # 'Number Killed': '',
    # 'Number Wounded': '',
    # 'Type of Gun': ''
}

for k, v in shooting_dic:
    resp = requests.get(base_url, params = params)
    response = resp.json()
    shooting_dic['date'].append(response['records']['fields']['date'])

,State,Year,Deaths,Population,Crude Rate
0,Texas,1999,2084,20558220,10.1
1,Texas,2000,2133,20851820,10.2
2,Texas,2001,2308,21319622,10.8
3,Texas,2002,2257,21690325,10.4
4,Texas,2003,2381,22030931,10.8


In [82]:
file_path = '../Resources/us_mass_shootings.csv'
us_mass_shooting = pd.read_csv(file_path, skiprows=7)
us_mass_shooting.tail()

,Date,City,State,Latitude,Longitude,Number killed,Number wounded,Assault weapon OR high-capacity magazine,Prohibited shooter,Killed intimate partner or family,Dangerous warning signs,Narrative,Total shot,Last updated
227,2020-03-15,Moncure,NC,35.588419,-79.016475,6,0,Under review,Under review,Under review,Under review,"A man shot and killed his wife, went to his ne...",6,2020-04-28T15:00:46.000Z
228,2020-03-15,Springfield,MO,37.210283,-93.236766,4,2,Under review,Under review,Under review,Under review,A man began firing shots while driving on the ...,6,2020-04-28T15:00:46.000Z
229,2020-01-22,Vanceboro,NC,35.233589,-77.110482,4,0,Under review,Under review,Under review,Under review,"A man shot and killed his wife, their three yo...",4,2020-06-25T21:27:44.000Z
230,2020-02-05,Indianapolis,IN,39.832988,-85.987690,4,0,Under review,Under review,Under review,Under review,An unknown suspect shot and killed four young ...,4,2020-06-25T21:27:50.000Z
231,2020-03-15,Springfield,MO,37.210283,-93.236766,4,2,Under review,Under review,Under review,Under review,A man began firing shots while driving on the ...,6,2020-06-25T21:27:50.000Z


In [83]:
columns = ['Date', 'City', 'State', 'Latitude', 'Longitude', 'Number killed', 'Number wounded', 'Assault weapon OR high-capacity magazine']
tx_vs_ca_df = us_mass_shooting.loc[(us_mass_shooting['State'] == 'TX')|(us_mass_shooting['State'] == 'CA'), columns]
tx_vs_ca_df.head()

,Date,City,State,Latitude,Longitude,Number killed,Number wounded,Assault weapon OR high-capacity magazine
0,2009-01-27,Wilmington,CA,33.783263,-118.254934,6,0,No
5,2009-03-21,East Oakland,CA,37.777688,-122.190217,4,1,Yes
7,2009-03-29,Santa Clara,CA,37.397970,-121.968242,5,1,No
15,2009-11-05,Fort Hood,TX,31.195006,-97.743503,13,32,Yes
22,2010-01-16,Bellville,TX,30.038827,-96.195849,5,0,Yes


In [ ]:
info_box_template = """
<dl>
<dt>Date</dt><dd>{Date}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>State</dt><dd>{State}</dd>
</dl>
"""
gmaps.configure(api_key=g_key)
figure_layout = {
    'width': '400px',
    'height': '420px',
    'border': '1px solid black',
    'padding': '1px'
}
gmaps.figure(center=(30.0, 31.0), zoom_level=2)
city_info = [info_box_template.format(**row) for index, row in tx_vs_ca_df.iterrows()]
locations = tx_vs_ca_df[["Latitude", "Longitude"]]
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=city_info)
fig.add_layer(marker_layer)
fig

In [84]:
tx_df = us_mass_shooting.loc[us_mass_shooting['State'] == 'TX']
tx_df.tail()

,Date,City,State,Latitude,Longitude,Number killed,Number wounded,Assault weapon OR high-capacity magazine,Prohibited shooter,Killed intimate partner or family,Dangerous warning signs,Narrative,Total shot,Last updated
188,2018-10-13,Taft,TX,27.9790,-97.3900,4,1,No,No,No,No,Two shooters were at a child’s birthday party ...,5,2019-11-12T14:45:16.000Z
198,2019-02-11,Blanchard,TX,30.7250,-95.0563,4,0,Under review,Under review,Under review,Under review,"A shooter shot and killed his wife, her grandp...",4,2020-04-28T15:00:46.000Z
209,2019-08-03,El Paso,TX,31.7767,-106.3850,23,23,Under review,Under review,Under review,Under review,A shooter traveled to a Walmart and shot 46 pe...,46,2020-04-28T15:00:46.000Z
211,2019-08-31,"Odessa, Midland",TX,31.9246,-102.2850,7,22,Under review,Under review,Under review,Under review,A shooter shot and killed seven people and inj...,29,2020-04-28T15:00:46.000Z
214,2019-09-29,Beaumont,TX,30.0473,-94.0855,4,0,Under review,Under review,Under review,Under review,A shooter shot and killed four people in an ap...,4,2020-04-28T15:00:46.000Z


In [ ]:
x_axis = tx_df['Date']                      
y_axis = tx_df['Number killed']
plt.bar(x_axis, y_axis)
plt.xlabel('Years')
plt.ylabel('Number of People Killed')
plt.title('Texas Mass Shooting Last Ten Years')
plt.show()

In [ ]:
ca_df = us_mass_shooting.loc[us_mass_shooting['State'] == 'CA']
ca_df.head()

In [ ]:
x_axis = ca_df['Date']                      
y_axis = ca_df['Number killed']
plt.bar(x_axis, y_axis)
plt.xlabel('Years')
plt.ylabel('Number of People Killed')
plt.title('California Mass Shooting Last Ten Years')
plt.show()

In [ ]:
state_comp = tx_vs_ca_df.loc[:, ['State', 'Number killed', 'Number wounded']]
state_comp = state_comp.groupby('State').sum()

state_comp

In [ ]:
state_comp.plot.bar()
plt.title('Texas vs California Mass Shooting Victims Last Ten Years')
plt.xlabel('State')
plt.ylabel('Total Number of People Killed and Wounded')
plt.show()